In [1]:
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50, decode_predictions
from keras import layers
from keras.models import Model, Sequential
from keras.callbacks.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [3]:
# create train, validation, and test generators from our image directory

datagen = ImageDataGenerator()

train_generator = datagen.flow_from_directory(
  directory=r"./train/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

val_generator = datagen.flow_from_directory(
  directory=r"./val/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

test_generator = datagen.flow_from_directory(
  directory=r"./test2/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=1,
  class_mode=None,
  shuffle=False,
  seed=42
)

Found 246 images belonging to 8 classes.
Found 120 images belonging to 8 classes.
Found 44310 images belonging to 8 classes.


## 1. Feature Extraction (20 points)

We will use a pretrained network (ResNet50) to extract features from the images. Then we will train a not-so-deep neural network on the features (X) and the labels (y). Finally we will evaluate our network. Please read [Vikas' tutorial on feature extraction](https://www.learnopencv.com/keras-tutorial-transfer-learning-using-pre-trained-models/) for code examples on feature extraction; we will use the same pipeline here.

🎒<font color='red'>(1 point)</font> Run the following code to load a [ResNet50 network](https://keras.rstudio.com/reference/application_resnet50.html) pretrained on the [ImageNet](http://www.image-net.org/) dataset. Note that we say `include_top=False` to exclude the last (or top) fully-connected layer which is responsible for classifying the labels in ImageNet; we want to use the earlier layers to extract features from our own dataset. Use `model.summary()` to inspect the ResNet50 architecture. This function should print out all the layers - their names, types, output shapes etc. Please include the printout in your submission.

In [4]:
# download the pre-trained ResNet50 model
resnet = keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))

# inspect the ResNet50 architecture
resnet.summary()

# observe that the last layer, conv5_block3_out (Activation), has output shape 7 * 7 * 2048
# let's save this dimension for later use
DIM = 7 * 7 * 2048

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

🎒<font color='red'>(2 points)</font> Use ResNet50 to extract features from the images. Print the shape of the extracted features.

👉 We use [`model.predict()`](https://keras.io/models/sequential/#predict_generator) to extract features. This function will return the output of the last layer of ResNet50, namely, conv5_block3_out (Activation). These are the features captured by the pretrained network.

👉 This step is slow when you use the full training set. Setting `verbose=1` can make a wait a bit easier.

In [5]:
# # use ResNet50 to extract features from the images
train_features = resnet.predict(train_generator, verbose=1) 
val_features = resnet.predict(val_generator, verbose=1)
test_features = resnet.predict(test_generator, verbose=1)

print(train_features.shape, val_features.shape, test_features.shape)

44310/44310 [==============================] - 21172s 478ms/step


KeyboardInterrupt: 

🎒<font color='red'>(1 point)</font> Observe that the features extracted by ResNet50 are 4D arrays. Following common practice, we want to reshape the features into 2D arrays, which will be the input X of our not-so-deep neural network classifier. Run the following code and print the shape of the 2D arrays.

In [6]:
# reshape the features to 2D arrays
train_X = train_features.reshape((-1, DIM))
val_X = val_features.reshape((-1, DIM))
test_X = test_features.reshape((-1, DIM))

print(train_X.shape, val_X.shape, test_X.shape)

NameError: name 'test_features' is not defined

🎒<font color='red'>(2 points)</font> Read the class labels from the generators. Print `test_labels`

In [7]:
# # read the class labels from the generators
train_labels = train_generator.classes
val_labels = val_generator.classes
test_labels = test_generator.classes

print(train_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


In [8]:
print(val_labels)

[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4
 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7
 7 7 7 7 7 7 7 7 7]


🎒<font color='red'>(2 points)</font> Observe that the labels are not one-hot encoded. We need to one-hot encode these labels and they will be the y that our model will predict.

In [9]:
print(test_labels)

[0 0 0 ... 7 7 7]


In [10]:
# # get one-hot encoding of labels
def get_one_hot(labels, nb_classes):
    res = np.eye(nb_classes)[np.array(labels).reshape(-1)]
    return res.reshape(list(labels.shape)+[nb_classes])

# # use get_one_hot()

NUM_CLASSES = 8
train_y = get_one_hot(train_labels,NUM_CLASSES)
val_y = get_one_hot(val_labels,NUM_CLASSES)
test_y = get_one_hot(test_labels,NUM_CLASSES)

🎒<font color='red'>(3 points)</font> Define neural network that contains a few layers. There is a three-layer example network below. Please modify this network - you can add layers, remove layers, or change some of the parameters. Please print out your model architecture using `model.summary()`.

In [17]:
# create our model: a not-so-deep neural network
model = Sequential()

# input layer takes arrays of shape (*, DIM)
# todo: please modify this network to define your own model
model.add(layers.Dense(500, activation = "relu", input_shape=(DIM,))) 
model.add(layers.Dropout(0.2, noise_shape=None, seed=100))
model.add(layers.Dense(NUM_CLASSES, activation = "softmax"))

# print out network architecture
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 500)               50176500  
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 4008      
Total params: 50,180,508
Trainable params: 50,180,508
Non-trainable params: 0
_________________________________________________________________


🎒<font color='red'>(1 point)</font> Compile your model using [`model.compile()`](https://keras.io/models/model/#compile). Choose `sgd` optimizer and `categorical_crossentropy` loss. Set `metrics = ["accuracy"]`

In [18]:
# # you need to compile the model before you can train it
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

🎒<font color='red'>(5 points)</font> Train the model you defined for about 10 epochs, or until the validation accuracy reaches 0.8 (whichever comes earlier). The model should be trained on `train_X` and `train_y`; it should be validated on `val_X` and `val_y`. Observe how the train accuracy and validation accuracy changes over epochs. Report (in a code comment) the best train and validation accuracy you've seen during training.

👉 Note that you can [check-point](https://keras.io/callbacks/#modelcheckpoint) model weights during training. After training, you can load back the weights of your best model.

In [19]:
# # save model weights while model is under training
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)

# # train the model for about 20 epochs
model.fit(
     x = train_X, 
     y = train_y, 
     epochs = 20,
     validation_data = (val_X,val_y),
     callbacks = [checkpoint]
)

Train on 217 samples, validate on 105 samples
Epoch 1/20
217/217 [==============================] - 2s 10ms/step - loss: 39.8605 - accuracy: 0.1429 - val_loss: 2.1462 - val_accuracy: 0.1429
Epoch 2/20
217/217 [==============================] - 2s 7ms/step - loss: 1.7819 - accuracy: 0.3456 - val_loss: 2.7282 - val_accuracy: 0.1333
Epoch 3/20
217/217 [==============================] - 1s 7ms/step - loss: 1.3106 - accuracy: 0.5023 - val_loss: 2.2370 - val_accuracy: 0.0857
Epoch 4/20
217/217 [==============================] - 2s 7ms/step - loss: 0.9381 - accuracy: 0.6820 - val_loss: 2.3881 - val_accuracy: 0.1333
Epoch 5/20
217/217 [==============================] - 2s 8ms/step - loss: 0.7414 - accuracy: 0.7650 - val_loss: 2.3360 - val_accuracy: 0.1048
Epoch 6/20
217/217 [==============================] - 2s 7ms/step - loss: 0.5517 - accuracy: 0.8065 - val_loss: 2.5148 - val_accuracy: 0.1905
Epoch 7/20
217/217 [==============================] - 1s 7ms/step - loss: 0.4293 - accuracy: 0.8756 

🎒<font color='red'>(2 points)</font> Using your best model, report the test accuracy.

In [20]:
# # load the weights of your best model
model.load_weights('model.h5')

In [21]:
# # predict test y labels (take a softmax on the model output)
pred_y = model.predict(test_X, batch_size=None, verbose=0, steps=None, callbacks = [checkpoint], max_queue_size=10, workers=1, use_multiprocessing=False)

In [22]:
# # measure test accuracy
score, acc = model.evaluate(test_X, test_y)
print('Test score:', score)
print('Test accuracy:', acc)

124/124 [==============================] - 0s 1ms/step
Test score: 2.157980772756761
Test accuracy: 0.07258064299821854


## 2. Fine-tuning (10 points)

Now we will explore the second technique: fine-tuning. We will append our own classification layers to the ResNet50 layers and train the entire network (from images to labels). We will start over from the generators.

In [11]:
# create train, validation, and test generators from our image directory

datagen = ImageDataGenerator()

train_generator = datagen.flow_from_directory(
  directory=r"./train/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=True,
  seed=42
)

val_generator = datagen.flow_from_directory(
  directory=r"./val/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=True,
  seed=42
)

test_generator = datagen.flow_from_directory(
  directory=r"./test2/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=1,
  class_mode=None,
  shuffle=False,
  seed=42
)

Found 246 images belonging to 8 classes.
Found 120 images belonging to 8 classes.
Found 44310 images belonging to 8 classes.


🎒<font color='red'>(3 points)</font> Append classification layers to ResNet50. Please modify the example layers below to define your own classification layers. Print the network architecture of your own model.

In [12]:
# # add custom prediction layers to ResNet50
x = resnet.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
predictions = layers.Dense(8, activation= 'softmax')(x)
model = Model(inputs = resnet.input, outputs = predictions)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

🎒<font color='red'>(1 point)</font> Compile your model using [`model.compile()`](https://keras.io/models/model/#compile). Choose `sgd` optimizer and `categorical_crossentropy` loss. Set `metrics = ["accuracy"]`

In [17]:
# # need to compile the model before training
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

🎒<font color='red'>(5 points)</font> Train the model you defined for about 10 epochs, or until the validation accuracy reaches 0.8 (whichever comes earlier). Checkpoint model weights during training. Report (in a code comment) the best train and validation accuracy you've seen during training.

In [14]:
# # train the model for about 10 epochs
STEP_SIZE_TRAIN=train_generator.n/train_generator.batch_size
STEP_SIZE_VAL=val_generator.n/val_generator.batch_size

In [9]:
# # save model weights while model is under training
model.fit_generator(generator= train_generator,
                     steps_per_epoch = 10,
                     validation_data = val_generator,
                     validation_steps= 5,
                     epochs=5
)

Epoch 1/5
10/10 [==============================] - 446s 45s/step - loss: 1.5190 - accuracy: 0.4984 - val_loss: 1.4038 - val_accuracy: 0.5592
Epoch 2/5
10/10 [==============================] - 668s 67s/step - loss: 0.4150 - accuracy: 0.9477 - val_loss: 0.8983 - val_accuracy: 0.6974
Epoch 3/5
10/10 [==============================] - 521s 52s/step - loss: 0.1533 - accuracy: 1.0000 - val_loss: 0.8244 - val_accuracy: 0.7500
Epoch 4/5
10/10 [==============================] - 747s 75s/step - loss: 0.1262 - accuracy: 0.9967 - val_loss: 0.8220 - val_accuracy: 0.8056
Epoch 5/5
10/10 [==============================] - 472s 47s/step - loss: 0.0775 - accuracy: 1.0000 - val_loss: 0.6208 - val_accuracy: 0.8158


In [10]:
model.save('new_model2.h5')

🎒<font color='red'>(1 point)</font> Using your best model, report the test accuracy.

In [18]:
# # load the weights of your best model
model.load_weights('new_model2.h5')

# # measure test accuracy
model.evaluate_generator(val_generator,steps=STEP_SIZE_VAL)

[0.9701080918312073, 0.824999988079071]

In [19]:
# # predict test y labels (take a softmax on the model output)
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,steps=STEP_SIZE_TEST,verbose=1)

44310/44310 [==============================] - 16819s 380ms/step


In [20]:
predicted_class_indices=np.argmax(pred,axis = 1)

In [21]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [22]:
import pandas as pd

filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results2.csv",index=False)